#### ID 2100

```Given the education levels and salaries of a group of individuals, find what is the average salary for each level of education.```

In [ ]:
%%sql
SELECT education, AVG(salary) AS avg_salary
FROM google_salaries
GROUP BY education

In [ ]:
df = google_salaries

df.groupby('education', as_index=False).agg(avg_salary=('salary', 'mean'))